# Multi Attributes Fractal Dimension

## Exploratory Data Analysis

Summarize about "SeisLab3.02" package. You can download from Mathworks [here](https://www.mathworks.com/matlabcentral/fileexchange/53109-seislab-3-02)

In [1]:
% Folder with 'SeisLab3.02' package
addpath(genpath('SeisLab3.02'))

% Folder with data
addpath(genpath('Data'))

In [2]:
data = read_segy_file('Data/Inline 900-903/seismic_amplitude_IL_900-903.segy', ...
                     {'headers', {'iline_no', 189, 4, 'n/a', 'In-line number'}})

Seismic data shifted since header "lag" is not identically zero.
Lag varies from 1300 to 1300

data = 

  struct with fields:

                      type: 'seismic'
                       tag: 'unspecified'
                      name: 'seismic_amplitude_IL_900-903 - shifted'
                      from: 'Data\Inline 900-903\seismic_amplitude_IL_900-903.segy'
               line_number: 1
               reel_number: 1
         traces_per_record: 1
                  cdp_fold: 1
                     first: 1300
                      last: 3300
                      step: 4
                     units: 'ms'
                    traces: [501x5132 double]
                      null: []
                   history: {2x4 cell}
               header_info: {13x3 cell}
                   headers: [13x5132 double]
    fp_format_of_segy_file: 'ibm'




In [3]:
data.traces(:, 100)


ans =

   1.0e+03 *

   -0.2449
    0.0700
    0.3643
    0.4591
    0.3147
    0.1509
    0.1677
    0.3139
    0.4047
    0.2904
   -0.0466
   -0.4181
   -0.5930
   -0.5475
   -0.3090
    0.0429
    0.3731
    0.5105
    0.3585
    0.0166
   -0.3615
   -0.5089
   -0.2981
    0.0123
    0.1394
    0.1476
    0.2171
    0.3240
    0.2675
   -0.0217
   -0.2957
   -0.3407
   -0.1421
    0.0961
    0.0957
   -0.0659
   -0.1363
   -0.0137
    0.1800
    0.2827
    0.2625
    0.1464
   -0.0483
   -0.1891
   -0.1700
   -0.0362
    0.0869
    0.0671
   -0.0110
   -0.0128
    0.0234
   -0.0520
   -0.1657
   -0.3103
   -0.4185
   -0.2935
    0.0518
    0.4063
    0.4866
    0.2955
    0.1579
    0.1487
    0.1070
   -0.0249
   -0.1539
   -0.1883
   -0.1497
   -0.1145
   -0.0273
    0.0633
   -0.0077
   -0.2620
   -0.5223
   -0.5411
   -0.1459
    0.4194
    0.7396
    0.7958
    0.6456
    0.3514
    0.0876
   -0.0487
   -0.1623
   -0.4377
   -0.7466
   -0.9134
   -0.8576
   -0.5216
   -0.0087

In [4]:
data.header_info


ans =

  13x3 cell array

    {'ds_seqno'  }    {'n/a'}    {'Trace sequence number within line'   }
    {'ffid'      }    {'n/a'}    {'Original Field record number'        }
    {'o_trace_no'}    {'n/a'}    {'Trace sequence number within orig...'}
    {'cdp'       }    {'n/a'}    {'CDP number'                          }
    {'seq_cdp'   }    {'n/a'}    {'Trace sequence number within CDP ...'}
    {'sou_x'     }    {'m'  }    {'X coordinate of source'              }
    {'sou_y'     }    {'m'  }    {'Y coordinate of source'              }
    {'cdp_x'     }    {'n/a'}    {'X-coordinate of CDP'                 }
    {'cdp_y'     }    {'n/a'}    {'Y-coordinate of CDP'                 }
    {'xline_no'  }    {'n/a'}    {'Cross-line number'                   }
    {'trc_type'  }    {'n/a'}    {'Trace type (1=live,2=dead,3=dummy...'}
    {'lag'       }    {'ms' }    {'Lag time between shot and recordi...'}
    {'iline_no'  }    {'n/a'}    {'In-line number'                      }




## Input data

In [5]:
% Definition of time window, above and below the horizon of interest.
TIME_ABOVE_HORIZON = 0;
TIME_BELOW_HORIZON = 20;

% Array of strings containing the names of the seismic attribute files.
files = {'Data/Inline 900-903/seismic_amplitude_IL_900-903.segy', ...
         'Data/Inline 900-903/instantaneous_amplitude_IL_900-903.segy'};
         
% Array of strings containing the titles of the input seismic attributes.
titlesFiles = {'Seismic amplitude', 'Instantaneous amplitude'};

In [23]:
% Definition of the weights for each attributes. It must be the same quantity as the input attributes and their sum must be
% equal to 1. These are used to control the contribution of each seismic attribute in the calculation of Fractal Dimension.
ALPHA = [0.5 0.5];

% File to save the calculation of Fractal Dimension.
fileFractalDim = {'EM_MER-U2_Fractal_Dimension.prn'};

% Number of traces to perform Quality Control (QC )of the calculations.
nTracesQc = 5;

## Load data 

### Surface Data

In [8]:
% Opening of the time surface of interest. The format must be X - Y - Time. Otherwise, it would generate an error.
% X-Y are coordinates.
assy = fopen('Data/EM_MER-U2.prn');

switch assy
    case -1
        warndlg('The surface file is not in the current directory', 'Error!');
    otherwise
        try
            reg = fscanf(assy, '%10f');
            surface = reshape(reg,3,[])';
        catch id
            errordlg('The file does not have the format X-Y-Time', 'Error!'); 
        end 
end

In [9]:
size(surface)


ans =

      504175           3




In [10]:
surface(1:10, :)


ans =

   1.0e+06 *

    0.3684    1.0184    0.0023
    0.3684    1.0186    0.0023
    0.3684    1.0189    0.0024
    0.3684    1.0190    0.0024
    0.3684    1.0195    0.0024
    0.3684    1.0196    0.0024
    0.3684    1.0197    0.0024
    0.3684    1.0198    0.0024
    0.3684    1.0201    0.0024
    0.3684    1.0203    0.0024




In [12]:
% Number of points contained within the surface.
nPointsSurface = size(surface,1);

### Seismic data

In [13]:
% Number of attributes to use in the algorithm
nAttributes = size(files, 2);

In [24]:
% Raise error dialog if the sum of the weights is different from one, or they differ from the number of input attributes.

if (sum(ALPHA) ~= 1.0)
    errordlg('The sum of weights are not equal to 1', 'Error!'); 
    
elseif (nAttributes ~= size(ALPHA,2))
    errordlg('The number of weights is not equal to the input attributes.', 'Error!'); 
    
end

The following data must be the same for each seismic volume.
- dt_sample: Sampling interval (microseconds).
- n_samples: Number of samples.
- n_traces: Number of traces.

In [ ]:
% If there was selected only one seismic volume, no verification need it.

if nAttributes >= 2
    [dt_sample,n_samples,n_traces] = Check_segy_data(files);

else
    [Segy_H] = ReadSegyHeader(files{1});
    dt_sample= Segy_H.dt/1000;
    n_samples= Segy_H.ns;
    n_traces = Segy_H.ntraces;

end